# import 3rd lib

In [1]:
from pynq import Overlay
from pynq.lib.iic import *
from pynq.lib.video import *
from time import sleep
from pynq import allocate
import matplotlib.pyplot as plt
import cv2
import math
import time
import os
import socket
import shutil

# Instantiate FPGA

In [2]:
ol = Overlay("design_1.bit")#_single_bak
ol.ip_dict

{'PL_PS_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xb4356b90>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'PL_PS_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S00_AXI',
  'parameters': {'C_S00_AXI_ADDR_WIDTH': '5',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF',
   'Component_Name': 'design_1_PL_PS_0_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 1136656384,
  'registers': {},
  'state': None,
  'type': 'user.org:user:PL_PS:1.0'},
 'axi_intc_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xb4356b90>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'axi_intc_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 's_axi',
  'parameters': {'C_ADDR_WIDTH': '32',
   'C_ASYNC_INTR': '0xFFFFFFF8',
   'C_BASEADDR': '0x41800000',
   'C_CASCADE_MASTER': '0',
   'C_DISABLE_SYNCHRONIZERS': '0',
   'C_ENABLE_ASYNC': '0',
   'C_EN_CASCADE_MODE': '0',
   'C_FAMILY':

In [3]:
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  #
local_addr = ("", 9442)  # 默认本机ip ，指定端口号9441
udp_socket.bind(local_addr)
dst_ip = "192.168.101.163"#目标ip
dst_port = 8000#目标端口

# init vdma

In [4]:
vdma0 = ol.axi_vdma_0
vdma1 = ol.axi_vdma_1
vdma1.readchannel.mode = VideoMode(640, 480, 24)
vdma0.readchannel.mode =vdma1.readchannel.mode
vdma0.writechannel.mode =vdma1.readchannel.mode
frame_out=allocate(shape=(480,640,3), dtype=np.uint8)
frame_out2=allocate(shape=(480,640,3), dtype=np.uint8)

In [5]:
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255, 0,0), 3)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0, 255, 0), 3)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0, 0, 255), 3)
    return img


def draw3dbox(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    for i, j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img

axis3dbox = np.float32([[0,0,0], [0,30,0], [30,30,0], [30,0,0],
                   [0,0,-30],[0,30,-30],[30,30,-30],[30,0,-30] ])

axis = np.float32([[50,0,0], [0,50,0], [0,0,50]]).reshape(-1,3)
stereo_cam_mt = np.array([[621.40807029,0,304.55052986],
                            [0,621.79033348,235.63191346],
                            [0,0,1]])
stereo_cam_ds = np.array([2.31802721e-01,-8.37438164e-01, -1.66196117e-03 , 3.67929303e-04, 3.00275469e-01])

object_2d_position = np.zeros((4, 2), np.float32)
object_3d_position = np.zeros((4, 3), np.float32)

object_3d_position[0]=np.array([0,0,0])#原点
object_3d_position[1]=np.array([75,0,0])#右
object_3d_position[2]=np.array([37,70,0])#上
object_3d_position[3]=np.array([37,35,0])#中心

In [8]:
vdma0.writechannel.stop()
vdma0.writechannel.start()

vdma0.readchannel.stop()
vdma0.readchannel.start()
while True:
    t1=time.time()
    maps = vdma0.readchannel.readframe()
    gray = cv2.cvtColor(maps,cv2.COLOR_BGR2GRAY) 
    _,bw = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    output = cv2.connectedComponentsWithStats(bw, 4, cv2.CV_32S)
    centroids = output[3]
    centroids = centroids[1::,:]
    t2=time.time()
    hh,ww = centroids.shape
    if hh == 3:
        object_2d_position[0, 0] =centroids[1,0]
        object_2d_position[0, 1] =centroids[1,1]
        object_2d_position[1, 0] =centroids[2,0]
        object_2d_position[1, 1] =centroids[2,1]
        object_2d_position[2, 0] =centroids[0,0]
        object_2d_position[2, 1] =centroids[0,1]
        object_2d_position[3, 0] =(centroids[0,0]+centroids[1,0]+centroids[2,0])/3
        object_2d_position[3, 1] =(centroids[0,1]+centroids[1,1]+centroids[2,1])/3
        _, rvec, tvec = cv2.solvePnP(object_3d_position, object_2d_position, stereo_cam_mt,stereo_cam_ds)  # 解算位姿
        distance = math.sqrt(tvec[0] ** 2 + tvec[1] ** 2 + tvec[2] ** 2)  # 计算距离
        rvec_matrix = cv2.Rodrigues(rvec)[0]  # 旋转向量->旋转矩阵
        proj_matrix = np.hstack((rvec_matrix, tvec))  # hstack: 水平合并
        eulerAngles = cv2.decomposeProjectionMatrix(proj_matrix)[6]  # 欧拉角
        thet_x, thet_y, thet_z = eulerAngles[0], eulerAngles[1], eulerAngles[2]
        #send_data = "Depth_distance: %.2fmm,3d pose: x: %.2f, y: %.2f, z: %.2f" % (distance, thet_x, thet_y, thet_z)
        send_data =" {\"key\":\"fpga_data\",\"rvec\":{\"x\":"+str(thet_y)+",\"y\":"+str(thet_x)+",\"z\":"+str(thet_z)+"},\"tvec\":{\"x\":"+str(tvec[0])+",\"y\":"+str(tvec[1])+",\"z\":"+str(tvec[2])+"}} "
        udp_socket.sendto(send_data.encode("utf-8"), (dst_ip, dst_port))
        cv2.putText(bw,'pos: '+ str(thet_x)+ ',' +str(thet_y)+','+str(thet_z) ,(20,50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1);
    gray3c = cv2.cvtColor(bw, cv2.COLOR_GRAY2RGB)    
    frame_out[:] = gray3c
    vdma0.writechannel.writeframe(frame_out)
    cv2.waitKey(20)

KeyboardInterrupt: 